# OCR

## With Unstructured.io

Hacer las peticiones al `http://localhost:8000`

Con el siguente comando se puede correr el contenedor de UNSTRUCTURED (con la opcion de `--rm` para que se elimine el contenedor al detenerlo):

```bash
docker run -p 8000:8000 -d --rm --name unstructured-api downloads.unstructured.io/unstructured-io/unstructured-api:latest --port 8000 --host 0.0.0.0
```

_Sin la opcion `--rm`:_
```bash
docker run -p 8000:8000 -d --name unstructured-api downloads.unstructured.io/unstructured-io/unstructured-api:latest --port 8000 --host 0.0.0.0
```

*Más imformación sobre la imagen: ([UnstructuredLoader | 🦜️🔗 Langchain](https://js.langchain.com/v0.2/docs/integrations/document_loaders/file_loaders/unstructured/))*

*Más imformación sobre el SDK de Python: [Process an individual file by using the Unstructured Python SDK - Unstructured](https://docs.unstructured.io/api-reference/api-services/sdk-python)*

In [ ]:
# Dependencies for unstructured

%pip install unstructured-client

Prueba con el siguiente bloque de codigo que hace la peticion al contenedor de UNSTRUCTURED, el cual se encarga de hacer el OCR a un archivo PDF; para despues guardar la salida en un archivo de texto.

```python
import os
import json
import unstructured_client
from unstructured_client.models import operations, shared

client = unstructured_client.UnstructuredClient(
    server_url="http://localhost:8000",
)

filename = "file.pdf"
with open(filename, "rb") as f:
    data = f.read()

req = operations.PartitionRequest(
    partition_parameters=shared.PartitionParameters(
        files=shared.Files(
            content=data,
            file_name=filename,
        ),
        strategy=shared.Strategy.HI_RES,
        languages=['eng'],
        split_pdf_page=True,            # If True, splits the PDF file into smaller chunks of pages.
        split_pdf_allow_failed=True,    # If True, the partitioning continues even if some pages fail.
        split_pdf_concurrency_level=15  # Set the number of concurrent request to the maximum value: 15.
    ),
)

try:
    res = client.general.partition(request=req)
    element_dicts = [element for element in res.elements]
    json_elements = json.dumps(element_dicts, indent=2)

    # Print the processed data.
    print(json_elements)

    # Write the processed data to a local file.
    with open("salida.txt", "w") as file:
        file.write(json_elements)
except Exception as e:
    print(e)
```

### Clase OCR

En la clase `OCR` se encuentra el metodo `get_ocr()` que en este caso se encarga de hacer el OCR a archivos de tipo PDF o rtf.
Tambien se cuenta con el metodo `get_dev_ocr()` que es el encargado de hacer la extracción de texto de archivos de texto plano.

**IMPORTANTE:**
Hay problemas con la concurrencia

In [ ]:
import json
from pathlib import Path
from typing import List, Dict
from unstructured_client import UnstructuredClient
from unstructured_client.models import operations, shared


class OCR:
    def __init__(self):
        self.unstructured_client = UnstructuredClient(server_url="http://localhost:8000")

    def get_ocr(self, file_path: str) -> List[Dict]:
        with open(file_path, "rb") as f:
            data = f.read()

        req = operations.PartitionRequest(
            partition_parameters=shared.PartitionParameters(
                files=shared.Files(
                    content=data,
                    file_name=file_path,
                ),
                strategy=shared.Strategy.AUTO,
                languages=['eng', 'spa'],
                split_pdf_page=False,            # If True, splits the PDF file into smaller chunks of pages.
                # split_pdf_allow_failed=True,    # If True, the partitioning continues even if some pages fail.
                # split_pdf_concurrency_level=15  # Set the number of concurrent request to the maximum value: 15.
            ),
        )
        try:
            res = self.unstructured_client.general.partition(request=req)
            element_dicts = [element for element in res.elements]
            json_elements = json.dumps(element_dicts, indent=2)

            # Print the processed data.
            print(json_elements)
            return element_dicts
        except Exception as e:
            print(e)

    def get_dev_ocr(self, file_path: str) -> Dict:
        file = Path(file_path)
        metadata = {"filetype": f'text/{file.suffix[1:]}' , "filename": file.name}
        text = file.read_text(encoding='utf-8')
        data = {
            'metadata': metadata, 
            'text': text
        }
        return [data]


---

## Without Unstructured.io

Unstructured.io es una herramienta poderosa, pero aparte de que tiene opciones de pago para mejor rendimiento, es bastante tardado hacer el OCR de documentos con elementos complejos (como imagenes, tablas, listas, etc.). Por lo que se encontro una alternativa que puede ser la libreria de ["pymupdf/PyMuPDF: PyMuPDF is a high performance Python library for data extraction, analysis, conversion & manipulation of PDF (and other) documents."](https://github.com/pymupdf/PyMuPDF) para hacer la extracción de texto a archivos PDF.

Los archivos PDF generalmente contienen imagenes, tablas, listas, etc. Por lo que primero se puede aplicar una capa de OCR al documento, para que la extracción de texto sea mas precisa como con un OCR de Unstructured.

Para agregar la capa de OCR a un archivo PDF se puede hacer con la herramienta de ["ocrmypdf/OCRmyPDF: OCRmyPDF adds an OCR text layer to scanned PDF files, allowing them to be searched"](https://github.com/ocrmypdf/OCRmyPDF).
OCRmyPDF ocupa la libreria de ["tesseract-ocr/tesseract: Tesseract Open Source OCR Engine"](https://github.com/tesseract-ocr/tesseract) para hacer el OCR a los archivos PDF. Por lo que primero se debe instalar la libreria de tesseract-ocr y despues instalar OCRmyPDF.

Como OCRmyPDF es una herramienta de linea de comandos, se puede hacer uso de la libreria de `subprocess` para hacer uso de OCRmyPDF en Python.

```python
import subprocess
import os

def ocr_pdf(input_pdf, output_pdf, language='eng+spa'):
    """
    Aplica OCR a un archivo PDF usando OCRmyPDF.

    :param input_pdf: Ruta al archivo PDF de entrada.
    :param output_pdf: Ruta al archivo PDF de salida.
    :param language: Idioma para OCR (por defecto 'eng+spa' para español).
    """
    try:
        # Construir el comando
        comando = [
            'ocrmypdf',
            '-l', language,
            '--force-ocr',
            '--jobs', '6',
            '--output-type', 'pdf',
            input_pdf,
            output_pdf
        ]

        # Ejecutar el comando
        subprocess.run(comando, check=True)
        print(f"OCR aplicado exitosamente a {input_pdf}. Salida: {output_pdf}")
    except subprocess.CalledProcessError as e:
        print(f"Error al aplicar OCR: {e}")

# Uso del script
if __name__ == "__main__":
    ruta_input = "ruta al archivo PDF de entrada"
    ruta_output = "ruta al archivo PDF de salida (puede ser el mismo archivo)"
    
    # Verificar si el archivo de entrada existe
    if os.path.exists(ruta_input):
        ocr_pdf(ruta_input, ruta_output)
    else:
        print(f"El archivo {ruta_input} no existe.")
```

### Clase OCR

En la clase `OCR` se encuentra el metodo `get_ocr()` que en este caso se encarga de hacer la extracción de texto de archivos PDF con la capa de OCR aplicada.
Este metodo manda llamar a la funcion `ocr_pdf()` que se encarga de aplicar la capa de OCR a un archivo PDF.
Tambien se cuenta con el mismo metodo `get_dev_ocr()` que es el encargado de hacer la extracción de texto de archivos de texto plano.

In [ ]:
# Dependencies for PyMuPDF

%pip install PyMuPDF

In [41]:
import subprocess
from pathlib import Path
from typing import List, Dict, Union
import pymupdf


class OCR:
    def ocr_pdf(self, input_pdf: Union[str, Path], output_pdf: Union[str, Path], language='eng+spa') -> None:
        """
        Aplica OCR a un archivo PDF usando OCRmyPDF.

        :param input_pdf: Ruta al archivo PDF de entrada.
        :param output_pdf: Ruta al archivo PDF de salida.
        :param language: Idioma para OCR (por defecto 'eng+spa' para ingles y español).
        """
        try:
            # Construir el comando
            comando = [
                'ocrmypdf',
                '-l', language,
                '--force-ocr',
                '--jobs', '6',
                '--output-type', 'pdf',
                input_pdf if isinstance(input_pdf, str) else input_pdf.resolve(),
                output_pdf if isinstance(output_pdf, str) else output_pdf.resolve()
            ]

            # Ejecutar el comando
            subprocess.run(comando, check=True)
            print(f"OCR aplicado exitosamente a {input_pdf}. Salida: {output_pdf}")
        except subprocess.CalledProcessError as e:
            print(f"Error al aplicar OCR: {e}")

    def get_ocr(self, file_path: str) -> List[Dict]:
        self.ocr_pdf(file_path, file_path)
        file = Path(file_path)
        elements = []
        metadata = {
            'filetype': 'application/pdf',
            'filename': file.name,
            'page_number': 0
        }
        doc = pymupdf.open(file.resolve())  # open a document
        for page in doc:
            metadata_copy = metadata.copy()  # Crear una copia del diccionario
            metadata_copy['page_number'] = page.number + 1
            elements.append({
                'metadata': metadata_copy,
                'text': page.get_text().encode('utf-8')
            })
        return elements
        
    def get_dev_ocr(self, file_path: str) -> List:
        file = Path(file_path)
        metadata = {"filetype": f'text/{file.suffix[1:]}' , "filename": file.name}
        text = file.read_text(encoding='utf-8')
        data = {
            'metadata': metadata, 
            'text': text
        }
        return [data]

#### Ejemplo de uso

In [42]:
ocr = OCR()

data = ocr.get_ocr('/Users/jorge-jrzz/Downloads/revista perros.pdf')
data

Start processing 6 pages concurrently
    2 page already has text! - rasterizing text and running OCR anyway
    3 page already has text! - rasterizing text and running OCR anyway
    4 page already has text! - rasterizing text and running OCR anyway
    5 page already has text! - rasterizing text and running OCR anyway
    6 page already has text! - rasterizing text and running OCR anyway
    7 page already has text! - rasterizing text and running OCR anyway
    7 Weighted average image DPI is 101.5, max DPI is 129.1. The discrepancy may indicate a high detail region on this page, but could also indicate a problem with the input PDF file. Page image will be rendered at 400.0 DPI.
    8 page already has text! - rasterizing text and running OCR anyway
    9 page already has text! - rasterizing text and running OCR anyway
   10 page already has text! - rasterizing text and running OCR anyway
   10 [tesseract] lots of diacritics - possibly poor OCR
Postprocessing...
Image optimization rat

OCR aplicado exitosamente a /Users/jorge-jrzz/Downloads/revista perros.pdf. Salida: /Users/jorge-jrzz/Downloads/revista perros.pdf


[{'metadata': {'filetype': 'application/pdf',
   'filename': 'revista perros.pdf',
   'page_number': 1},
  'text': b'Revista veterinaria profesional\n(a de animales de compa\xc3\xb1\xc3\xada\nN\xc2\xb0 130 - Octubre 2014\n5 (2014) - A\xc3\xb1o XXII\nEl laboratorio en tu clinica Cn de\nwww.ciab.es | rodrigo@ciab.es | 91 3613314 Investigacion y\noslada 12. Baio Drcha. 28028 Madrid Analisis Biologicos\n'},
 {'metadata': {'filetype': 'application/pdf',
   'filename': 'revista perros.pdf',
   'page_number': 2},
  'text': b'18\na GENETICA DE PERROS Y GATOS: PATOLOGIAS HEREDITARIAS\nCANIf-FELIf | Y OTROS ASPECTOS DE INTER\xc3\x89S EN LA CL\xc3\x8dNICA VETERINARIA :\nOrigen y diversidad\nde la especie canina\nDunner S, Ca\xc3\xb1\xc3\xb3n J\nLaboratorio de Gen\xc3\xa9tica. Dpto. de Producci\xc3\xb3n Animal. Facultad de Veterinaria, UCM.\nDesde el inicio del proceso de domesticacion del perro, hace unos 11.000-16.000\nanos a partir de poblaciones de lobo, el ancestro salvaje del perro, el patro